### features:
* spectogram coef
* zero crossing rate
* selected chromogram values
* selected tempogram values
* wavelet levels

In [1]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, recall_score
import matplotlib.pyplot as plt
import itertools

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
path = "/Users/ecem/Desktop/phonocardiogram/data/AV"

## Load data

Here, our target is to predict disease of patients with multiple diseases. So; our y will be disease column of two column indicatin geach patients disease.

In [5]:
present_df = pd.read_csv(path + "/present/present_AV_p_features.csv", index_col = 0) #1245, 18752
present_df = present_df.sample(n=1080, random_state=42)
present_df

,freq_0,freq_1,freq_2,freq_3,freq_4,freq_5,freq_6,freq_7,freq_8,freq_9,...,tempogram_165,tempogram_166,tempogram_167,tempogram_168,tempogram_173,tempogram_180,tempogram_181,tempogram_182,tempogram_184,tempogram_191
735,5.361274e-11,7.572183e-11,7.536700e-11,7.469895e-11,7.380934e-11,7.285138e-11,7.195599e-11,7.131806e-11,7.110692e-11,7.144569e-11,...,7.372882e-18,1.396782e-17,-1.302083e-17,-5.063569e-19,-5.247270e-19,1.331894e-17,1.530888e-17,-1.215481e-17,7.645321e-18,2.083399e-17
442,4.064968e-11,5.856189e-11,5.964342e-11,6.022107e-11,6.016372e-11,5.943264e-11,5.811892e-11,5.650605e-11,5.510679e-11,5.464973e-11,...,2.106225e-09,6.915017e-10,1.575394e-10,1.537404e-11,-1.484337e-17,1.058228e-17,1.849605e-17,-1.131010e-17,9.972253e-18,2.025733e-17
1117,4.087155e-11,5.819980e-11,5.900241e-11,6.004508e-11,6.116654e-11,6.223143e-11,6.314993e-11,6.388121e-11,6.441495e-11,6.471712e-11,...,-1.765057e-18,5.419016e-18,-8.399998e-18,-1.024720e-18,-6.429164e-18,1.120789e-17,1.615662e-17,-1.031348e-18,1.149566e-17,1.260996e-17
333,2.979893e-10,4.317041e-10,4.599245e-10,5.010812e-10,5.506623e-10,6.048916e-10,6.604523e-10,7.143498e-10,7.638995e-10,8.067737e-10,...,-1.084998e-18,8.856700e-18,-1.832179e-17,-2.804687e-18,-1.159928e-17,7.620874e-18,1.523931e-17,-6.843847e-18,1.065373e-17,1.655499e-17
327,2.129128e-10,2.992713e-10,2.934330e-10,2.836717e-10,2.703796e-10,2.542287e-10,2.364742e-10,2.212936e-10,2.250312e-10,2.362572e-10,...,1.438546e-09,4.888467e-10,1.129202e-10,1.103460e-11,-8.314551e-18,8.189450e-18,1.391022e-17,-1.388869e-17,-4.691424e-18,2.189974e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,3.681836e-10,5.271169e-10,5.440504e-10,5.690364e-10,5.994921e-10,6.326890e-10,6.660306e-10,6.972113e-10,7.242811e-10,7.456742e-10,...,2.249124e-10,5.246009e-11,8.664748e-12,6.593652e-13,-9.645507e-18,4.209801e-18,2.050888e-17,-5.174531e-18,5.733066e-18,1.861149e-17
293,1.927356e-10,2.761133e-10,2.830122e-10,2.894615e-10,2.940086e-10,2.962540e-10,2.963784e-10,2.949262e-10,2.926784e-10,2.924980e-10,...,4.617464e-09,1.332110e-09,2.610449e-10,2.275927e-11,-8.789493e-18,4.382196e-18,2.121432e-17,-1.726316e-18,8.477385e-18,1.677950e-17
1242,2.886341e-10,4.070661e-10,4.033971e-10,3.969121e-10,3.879849e-10,3.776499e-10,3.672374e-10,3.579410e-10,3.507390e-10,3.466603e-10,...,4.953307e-09,1.643199e-09,3.728994e-10,3.603946e-11,-1.049532e-17,1.875301e-17,3.001944e-17,-8.987125e-18,2.075392e-17,2.501257e-17
400,8.245434e-11,1.186469e-10,1.228147e-10,1.275198e-10,1.320275e-10,1.358709e-10,1.387439e-10,1.404578e-10,1.409093e-10,1.400814e-10,...,8.835005e-19,5.060995e-18,-1.027160e-17,-4.046912e-18,-1.065599e-17,6.063483e-18,1.625138e-17,1.168057e-18,-3.588671e-18,1.058701e-17


In [4]:
absent_df = pd.read_csv(path + "/absent/absent_AV_p_features.csv", index_col = 0) #6890, 18752
absent_df

,freq_0,freq_1,freq_2,freq_3,freq_4,freq_5,freq_6,freq_7,freq_8,freq_9,...,tempogram_165,tempogram_166,tempogram_167,tempogram_168,tempogram_173,tempogram_180,tempogram_181,tempogram_182,tempogram_184,tempogram_191
0,9.032866e-11,1.283307e-10,1.273762e-10,1.246309e-10,1.201637e-10,1.142205e-10,1.071923e-10,9.958826e-11,9.202662e-11,8.600805e-11,...,1.229573e-08,3.539518e-09,6.913353e-10,6.015870e-11,-4.425509e-18,1.527628e-17,3.069961e-17,3.360353e-18,1.392965e-17,2.439336e-17
1,8.539332e-11,1.200104e-10,1.177622e-10,1.140818e-10,1.091471e-10,1.033975e-10,9.780705e-11,9.387825e-11,9.239269e-11,9.306054e-11,...,4.168072e-10,9.191343e-11,1.751558e-11,1.681676e-12,-1.903726e-17,1.009219e-17,2.060304e-17,-8.288342e-18,-2.530324e-18,1.477412e-17
2,6.311897e-11,8.902436e-11,8.830606e-11,8.712377e-11,8.552773e-11,8.359623e-11,8.144592e-11,7.934172e-11,7.727474e-11,7.657807e-11,...,1.407631e-10,4.028191e-11,7.858989e-12,6.858554e-13,-3.553411e-18,9.669628e-18,2.088307e-17,-8.270446e-18,4.801591e-18,2.178446e-17
3,6.343676e-11,9.022266e-11,9.124019e-11,9.224844e-11,9.295056e-11,9.321291e-11,9.301599e-11,9.241415e-11,9.151151e-11,9.047624e-11,...,5.123989e-09,1.636423e-09,3.547727e-10,3.318718e-11,-1.173998e-17,2.113700e-17,3.271036e-17,-5.766655e-18,6.897437e-18,1.759136e-17
4,7.630806e-11,1.079990e-10,1.080264e-10,1.076320e-10,1.065868e-10,1.048177e-10,1.024159e-10,1.003597e-10,1.000656e-10,9.960670e-11,...,1.488253e-09,5.085275e-10,1.179416e-10,1.154484e-11,-2.253705e-17,1.134327e-17,3.033557e-17,-1.462569e-18,1.988642e-17,2.560940e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6975,4.391115e-11,6.325533e-11,6.596380e-11,6.906291e-11,7.179823e-11,7.381084e-11,7.502210e-11,7.552006e-11,7.542620e-11,7.479751e-11,...,1.610333e-08,5.381846e-09,1.215283e-09,1.165740e-10,-1.364235e-17,1.561749e-17,2.741938e-17,-6.581439e-18,1.704671e-17,2.236864e-17
6976,6.021307e-11,8.525027e-11,8.540722e-11,8.544461e-11,8.529221e-11,8.496842e-11,8.454631e-11,8.414250e-11,8.394189e-11,8.404234e-11,...,6.409633e-09,1.985497e-09,4.324786e-10,4.108762e-11,-1.367223e-17,5.665199e-18,2.509268e-17,-8.479062e-18,5.454273e-18,1.684958e-17
6977,4.200291e-11,5.951789e-11,5.957082e-11,5.933758e-11,5.883054e-11,5.856920e-11,5.882910e-11,5.902975e-11,5.904306e-11,5.891823e-11,...,1.495706e-08,4.869521e-09,1.077951e-09,1.023359e-10,-1.660658e-17,3.492718e-17,3.607055e-17,-7.115761e-18,2.491467e-17,3.799787e-17
6978,3.381760e-11,4.864368e-11,4.980425e-11,5.108621e-11,5.242817e-11,5.394787e-11,5.591244e-11,5.805302e-11,6.023821e-11,6.247719e-11,...,1.562378e-09,5.046506e-10,1.114649e-10,1.058987e-11,-1.277173e-17,1.912307e-17,3.395590e-17,-9.225937e-18,1.692093e-17,3.087130e-17


In [ ]:
absent_df = absent_df.sample(n=1080, )

### create labels

In [5]:
present_labels = pd.DataFrame(np.ones((1245)),columns = ["label"])
present_labels

,label
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1240,1.0
1241,1.0
1242,1.0
1243,1.0


In [6]:
#present_df = pd.concat([present_df, present_labels], axis=1)
#present_df

In [7]:
absent_labels = pd.DataFrame(np.zeros((6980)), columns = ["label"])
absent_labels

,label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
6975,0.0
6976,0.0
6977,0.0
6978,0.0


In [8]:
#absent_df = pd.concat([absent_df, absent_labels], axis=1)
#absent_df

## concat features

In [9]:
df = pd.concat([present_df, absent_df], axis = 0) #8225, 18753
df

,freq_0,freq_1,freq_2,freq_3,freq_4,freq_5,freq_6,freq_7,freq_8,freq_9,...,tempogram_165,tempogram_166,tempogram_167,tempogram_168,tempogram_173,tempogram_180,tempogram_181,tempogram_182,tempogram_184,tempogram_191
0,4.053941e-11,5.722774e-11,5.674817e-11,5.604892e-11,5.585778e-11,5.712218e-11,6.096665e-11,6.666104e-11,7.312381e-11,7.987906e-11,...,3.694971e-11,1.134680e-11,2.389866e-12,2.203077e-13,-8.122705e-18,8.131668e-18,2.377722e-17,9.874380e-21,3.277308e-18,1.594158e-17
1,7.029058e-11,1.004992e-10,1.032565e-10,1.067600e-10,1.103392e-10,1.136039e-10,1.163349e-10,1.183813e-10,1.195946e-10,1.198336e-10,...,2.385996e-08,7.722310e-09,1.684974e-09,1.577951e-10,4.220379e-20,2.353653e-17,4.041855e-17,3.897346e-18,2.307725e-17,3.545423e-17
2,6.242156e-11,8.838291e-11,8.841429e-11,8.813666e-11,8.771334e-11,8.792236e-11,8.855698e-11,8.896638e-11,8.933381e-11,8.922699e-11,...,1.597677e-09,5.115552e-10,1.114111e-10,1.047179e-11,-1.754692e-17,1.287416e-17,2.539911e-17,-7.691983e-18,7.988967e-19,1.729207e-17
3,5.658126e-11,8.147643e-11,8.364837e-11,8.546564e-11,8.658269e-11,8.682239e-11,8.612547e-11,8.458010e-11,8.250057e-11,8.019542e-11,...,1.609346e-08,4.978872e-09,1.062980e-09,9.910194e-11,-8.994226e-18,3.161606e-17,3.944629e-17,4.191656e-18,1.564245e-17,2.644279e-17
4,6.675927e-11,9.572394e-11,9.878655e-11,1.025290e-10,1.061944e-10,1.092672e-10,1.114632e-10,1.127096e-10,1.129850e-10,1.121774e-10,...,1.053892e-08,3.516443e-09,7.916528e-10,7.582421e-11,-1.561858e-17,1.496655e-17,2.920759e-17,-7.881559e-18,1.673272e-17,1.686190e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6975,4.391115e-11,6.325533e-11,6.596380e-11,6.906291e-11,7.179823e-11,7.381084e-11,7.502210e-11,7.552006e-11,7.542620e-11,7.479751e-11,...,1.610333e-08,5.381846e-09,1.215283e-09,1.165740e-10,-1.364235e-17,1.561749e-17,2.741938e-17,-6.581439e-18,1.704671e-17,2.236864e-17
6976,6.021307e-11,8.525027e-11,8.540722e-11,8.544461e-11,8.529221e-11,8.496842e-11,8.454631e-11,8.414250e-11,8.394189e-11,8.404234e-11,...,6.409633e-09,1.985497e-09,4.324786e-10,4.108762e-11,-1.367223e-17,5.665199e-18,2.509268e-17,-8.479062e-18,5.454273e-18,1.684958e-17
6977,4.200291e-11,5.951789e-11,5.957082e-11,5.933758e-11,5.883054e-11,5.856920e-11,5.882910e-11,5.902975e-11,5.904306e-11,5.891823e-11,...,1.495706e-08,4.869521e-09,1.077951e-09,1.023359e-10,-1.660658e-17,3.492718e-17,3.607055e-17,-7.115761e-18,2.491467e-17,3.799787e-17
6978,3.381760e-11,4.864368e-11,4.980425e-11,5.108621e-11,5.242817e-11,5.394787e-11,5.591244e-11,5.805302e-11,6.023821e-11,6.247719e-11,...,1.562378e-09,5.046506e-10,1.114649e-10,1.058987e-11,-1.277173e-17,1.912307e-17,3.395590e-17,-9.225937e-18,1.692093e-17,3.087130e-17


## concat labels

In [10]:
labels = pd.concat([present_labels, absent_labels], axis = 0) #8225, 18753
labels

,label
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
6975,0.0
6976,0.0
6977,0.0
6978,0.0


## Without threshold

In [11]:
X_train_, X_valid_, y_train_, y_valid_ = train_test_split(df, labels , train_size=0.8, test_size=0.2,
                                                                random_state=42)

In [12]:
import xgboost as xgb

clf = xgb.XGBClassifier(objective= "binary:logistic", eval_metric = "logloss")


In [ ]:
clf.fit(X_train_, y_train_)


In [ ]:
yhat = clf.predict(X_valid_)

In [ ]:
yhat.shape

In [ ]:
y_valid_.shape

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_valid_, yhat)
print("accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import recall_score
recall = recall_score(y_valid_, yhat, average= 'weighted')
print("recall: %.2f%%" % (recall * 100.0))


In [ ]:
from sklearn.metrics import precision_score
precision = precision_score(y_valid_, yhat, average= 'weighted')
print("precision: %.2f%%" % (precision * 100.0))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf, X_valid_, y_valid_) 

In [ ]:
names =["absent", "present"]
print(classification_report(y_valid_, yhat, target_names = names))